In [1]:
import pandas as pd

base = pd.read_csv('dataset.csv')

In [2]:
import numpy as np

previsores = base.iloc[:1400000,0:20].values
previsores = np.reshape(previsores, (2000, 700, 20))
print(previsores.shape)

classes = base.iloc[:1400000, 28:31].values
classes = np.reshape(classes, (2000, 700, 3))
print(classes.shape)

(2000, 700, 20)
(2000, 700, 3)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNLSTM, Bidirectional, Masking

In [5]:
def criarRede():
    model = Sequential()
  
    #model.add(Masking(mask_value = 0, input_shape = (700, 20)))
  
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
  
    model.add(Dense(3, activation = 'softmax'))
  
    model.compile(optimizer = 'adam', metrics = ['acc'], loss='categorical_crossentropy')
  
    return model

In [6]:
accu = []
precisions = []
recalls = []
f1 = []

In [7]:
from sklearn.metrics import balanced_accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train, epochs = 100, verbose = 1, batch_size = 32)
  
    predicted = model.predict(x_test)
  
    y_teste = []
    predict = []
  
    predicted = np.reshape(predicted, (predicted.shape[0] * predicted.shape[1], 3))
    y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], 3))
    x_test = np.reshape(x_test, (x_test.shape[0] * x_test.shape[1], 20))

    for i in range(len(x_test)):
        cont = 0
        for j in range(len(x_test[i])):
            cont += x_test[i][j]
        if cont != 0:
            y_teste.append(y_test[i])
            predict.append(predicted[i])
    
    y_teste = np.asarray(y_teste)
    predict = np.asarray(predict)

    predicted = predict
    y_test = y_teste

    predicted = np.argmax(predicted, axis=1)
    y_test = np.argmax(y_test, axis=1)

    accu.append(balanced_accuracy_score(y_test, predicted)) 
    precisions.append(precision_score(y_test, predicted, average = 'weighted'))
    recalls.append(recall_score(y_test, predicted, average = 'weighted'))
    f1.append(f1_score(y_test, predicted, average = 'weighted'))
    
    print(classification_report(y_test, predicted))
    print('Acurácia')
    print(balanced_accuracy_score(y_test, predicted))
    print('Precisao')
    print(precision_score(y_test, predicted, average = 'weighted'))
    print('Recall')
    print(recall_score(y_test, predicted, average = 'weighted'))
    print('F1')
    print(f1_score(y_test, predicted, average = 'weighted'))
    print(confusion_matrix(y_test, predicted))

In [8]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede()
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classes[train_index],
                           previsores[test_index], classes[test_index])

W0820 10:44:50.348475  7992 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0820 10:44:50.358447  7992 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0820 10:44:50.359444  7992 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py

TRAIN: [   3    4    5 ... 1997 1998 1999] TEST: [   0    1    2    6   11   13   16   21   33   34   44   49   50   56
   59   60   74   77   79   83   88  104  113  117  121  125  126  136
  151  156  160  162  163  168  173  176  177  182  184  186  189  200
  203  216  224  228  232  238  239  247  261  266  280  282  284  289
  294  297  303  313  315  316  319  329  333  344  353  362  364  373
  374  376  378  379  385  392  393  395  398  410  419  424  431  437
  441  442  444  445  469  473  484  489  492  496  497  504  505  509
  512  518  519  523  524  525  540  544  549  550  555  556  561  570
  571  572  576  577  589  594  595  596  597  601  603  607  610  614
  627  628  636  637  643  648  656  677  691  697  704  705  719  721
  730  731  741  742  746  751  756  758  762  770  772  777  778  783
  788  789  794  797  799  805  817  820  821  828  830  845  850  851
  853  857  858  865  872  880  882  885  896  901  909  914  916  919
  920  921  925  932  934  9

W0820 10:44:52.048926  7992 deprecation.py:323] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1600/1600 [==============================] - 20s 12ms/sample - loss: 0.3682 - acc: 0.7865
Epoch 2/100
1600/1600 [==============================] - 17s 10ms/sample - loss: 0.3324 - acc: 0.8451
Epoch 3/100
1600/1600 [==============================] - 17s 10ms/sample - loss: 0.3149 - acc: 0.8565
Epoch 4/100
1600/1600 [==============================] - 17s 10ms/sample - loss: 0.3059 - acc: 0.8612
Epoch 5/100
1600/1600 [==============================] - 17s 10ms/sample - loss: 0.3003 - acc: 0.8644
Epoch 6/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2970 - acc: 0.8656
Epoch 7/100
1600/1600 [==============================] - 17s 10ms/sample - loss: 0.2944 - acc: 0.8671
Epoch 8/100
1600/1600 [==============================] - 17s 10ms/sample - loss: 0.2924 - acc: 0.8686
Epoch 9/100
1600/1600 [==============================] - 17s 10ms/sample - loss: 0.2943 - acc: 0.8673
Epoch 10/100
1600/1600 [==============================] - 17s 10ms/sample - loss: 

1600/1600 [==============================] - 17s 10ms/sample - loss: 0.1158 - acc: 0.9506
Epoch 81/100
1600/1600 [==============================] - 17s 10ms/sample - loss: 0.1163 - acc: 0.9502
Epoch 82/100
1600/1600 [==============================] - 17s 10ms/sample - loss: 0.1153 - acc: 0.9512
Epoch 83/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1151 - acc: 0.9508
Epoch 84/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1122 - acc: 0.9518
Epoch 85/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1114 - acc: 0.9524
Epoch 86/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1107 - acc: 0.9529
Epoch 87/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1099 - acc: 0.9533
Epoch 88/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1087 - acc: 0.9535
Epoch 89/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.10

...

Epoch 1/100
1600/1600 [==============================] - 18s 11ms/sample - loss: 0.3710 - acc: 0.7293
Epoch 2/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.3341 - acc: 0.8427
Epoch 3/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.3117 - acc: 0.8590
Epoch 4/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.3046 - acc: 0.8625
Epoch 5/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2998 - acc: 0.8645
Epoch 6/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2977 - acc: 0.8661
Epoch 7/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2940 - acc: 0.8674
Epoch 8/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2927 - acc: 0.8672
Epoch 9/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2893 - acc: 0.8683
Epoch 10/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 

1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1107 - acc: 0.9502
Epoch 81/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1102 - acc: 0.9508
Epoch 82/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1093 - acc: 0.9516
Epoch 83/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1089 - acc: 0.9514
Epoch 84/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1081 - acc: 0.9513
Epoch 85/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1063 - acc: 0.9528
Epoch 86/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1053 - acc: 0.9533
Epoch 87/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1050 - acc: 0.9535
Epoch 88/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1049 - acc: 0.9527
Epoch 89/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.10

1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2034 - acc: 0.9120
Epoch 35/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2016 - acc: 0.9126
Epoch 36/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1992 - acc: 0.9142
Epoch 37/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1947 - acc: 0.9157
Epoch 38/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1881 - acc: 0.9186
Epoch 39/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1840 - acc: 0.9204
Epoch 40/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1807 - acc: 0.9212
Epoch 41/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1792 - acc: 0.9224
Epoch 42/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1748 - acc: 0.9237
Epoch 43/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.17

Epoch 1/100
1600/1600 [==============================] - 18s 11ms/sample - loss: 0.3690 - acc: 0.7787
Epoch 2/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.3392 - acc: 0.8405
Epoch 3/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.3129 - acc: 0.8579
Epoch 4/100
1600/1600 [==============================] - 17s 11ms/sample - loss: 0.3050 - acc: 0.8621
Epoch 5/100
1600/1600 [==============================] - 17s 10ms/sample - loss: 0.3002 - acc: 0.8646
Epoch 6/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2970 - acc: 0.8667
Epoch 7/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2954 - acc: 0.8669
Epoch 8/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2915 - acc: 0.8689
Epoch 9/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.2879 - acc: 0.8698
Epoch 10/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 

1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1104 - acc: 0.9523
Epoch 81/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1089 - acc: 0.9526
Epoch 82/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1085 - acc: 0.9535
Epoch 83/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1067 - acc: 0.9535
Epoch 84/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1070 - acc: 0.9533
Epoch 85/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1052 - acc: 0.9543
Epoch 86/100
1600/1600 [==============================] - 17s 10ms/sample - loss: 0.1060 - acc: 0.9533
Epoch 87/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1046 - acc: 0.9542
Epoch 88/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.1038 - acc: 0.9544
Epoch 89/100
1600/1600 [==============================] - 17s 10ms/sample - loss: 0.10

In [9]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_24 (Bidirectio (None, 700, 200)          97600     
_________________________________________________________________
dropout_24 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_25 (Bidirectio (None, 700, 200)          241600    
_________________________________________________________________
dropout_25 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_26 (Bidirectio (None, 700, 200)          241600    
_________________________________________________________________
dropout_26 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_27 (Bidirectio (None, 700, 200)         

In [10]:
print('Acurácias total')
print(accu)
accu = np.array(accu)
print(accu.mean())
print('Precision total')
print(precisions)
precisions = np.array(precisions)
print(precisions.mean())
print('Recalls total')
print(recalls)
recalls = np.array(recalls)
print(recalls.mean())
print('F1 total')
print(f1)
f1 = np.array(f1)
print(f1.mean())

Acurácias total
[0.680328345106108, 0.6780512116868812, 0.6878495638982721, 0.6659930019958127, 0.6782691195276168]
0.6780982484429382
Precision total
[0.6931110677420136, 0.6882573969288281, 0.7036790688131369, 0.6800400257973842, 0.690721368810973]
0.6911617856184671
Recalls total
[0.6937317342560283, 0.6890500816282639, 0.7046029484458143, 0.6811767025661731, 0.689979513847851]
0.6917081961488261
F1 total
[0.6929647361330042, 0.6885019958791311, 0.7032244447763936, 0.6798570029510789, 0.6897716334343099]
0.6908639626347834
